In [1]:
from BiGCN_Trainer import *

Using backend: pytorch


In [6]:
import os

In [2]:
os.environ['DaMSTF_PATH'] = '/data01/home/scy0798/run/MLLu/da-mstf_-paper_-codes'

In [7]:
    data_dir = os.path.abspath(os.path.join(os.environ['DaMSTF_PATH'], "../pheme-rnr-dataset/"))
    events_list = ['charliehebdo',  'ferguson',  'germanwings-crash',  'ottawashooting',  'sydneysiege']
    # for domain_ID in range(5):
    domain_ID = 0
    source_events = [os.path.join(data_dir, dname)
                     for idx, dname in enumerate(events_list) if idx != domain_ID]
    target_events = [os.path.join(data_dir, events_list[domain_ID])]
    test_event_name = events_list[domain_ID]

In [4]:
    tfidf_vectorizer, tr, _, dev, te = load_data_and_TfidfVectorizer(
        source_events, target_events, 0, unlabeled_ratio=-1
    )

load len:  1143
load len:  469
load len:  890
load len:  1221
load len:  2079
data length :  3723/0/415/1664


3723it [00:02, 1433.60it/s]
3723it [00:03, 1219.38it/s]
415it [00:00, 3553.79it/s]
415it [00:00, 1183.34it/s]
1664it [00:00, 3588.90it/s]
1664it [00:01, 1212.75it/s]


In [5]:
    model = obtain_BiGCN(tfidf_vectorizer)
    dev_eval = BiGCNEvaluator(dev, batch_size=20)
    te_eval = BiGCNEvaluator(te, batch_size=20)

self.embedding.weight.requires_grad False
requires_grad = True
OOV Count: 3529
OOV Ratio: 0.17261788299745648


NameError: name 'log_dir' is not defined

In [16]:
from torch.utils.data import Dataset
class BiGCNTrainer(RumorBaseTrainer):
    def __init__(self, log_dir, tokenizer:TfidfVectorizer, **kwargs):
        super(BiGCNTrainer, self).__init__()
        for k, v in kwargs.items():
            if not hasattr(self, k):
                raise Exception(f"Attribute '{k}' is not a valid attribute of DaMSTF")
            self.__setattr__(k, v)

        self.running_dir = log_dir
        if not os.path.exists(self.running_dir):
            os.system(f"mkdir {self.running_dir}")
        self.tokenizer = tokenizer
        self.device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

    def collate_fn(self, items):
        device = torch.device("cuda") if torch.cuda.is_available() else torch.device('cpu')
        tfidf_arr = torch.cat(
            [item[0] for item in items],
            dim=0
        )
        TD_graphs = [item[1] for item in items]
        BU_graphs = [item[2] for item in items]
        labels = [item[3] for item in items]
        topic_labels = [item[4] for item in items]
        num_nodes = [g.num_nodes() for g in TD_graphs]
        big_g_TD = dgl.batch(TD_graphs)
        big_g_BU = dgl.batch(BU_graphs)
        A_TD = big_g_TD.adjacency_matrix().to_dense().to(device)
        A_BU = big_g_BU.adjacency_matrix().to_dense().to(device)
        return tfidf_arr, num_nodes, A_TD, A_BU, \
               torch.tensor(labels), torch.tensor(topic_labels)

    def trainset2trainloader(self, dataset:Dataset, shuffle=False, batch_size=32):
        return self.dataset2dataloader(dataset, shuffle, batch_size)
    
    def dataset2dataloader(self, dataset:FastBiGCNDataset, shuffle=False, batch_size=32):
        if shuffle:
            idxs = random.sample(range(len(dataset)), len(dataset))*2
        else:
            idxs = [*range(len(dataset))]*2
        @background(max_prefetch=5)
        def dataloader():
            for start in range(0, len(dataset), batch_size):
                batch_idxs = idxs[start:start+batch_size]
                items = [dataset[index] for index in batch_idxs]
                yield self.collate_fn(items)
        return dataloader()

In [17]:
log_dir = f"./{test_event_name}/"
trainer = BiGCNTrainer(log_dir, tfidf_vectorizer, model_rename=True)

In [18]:
    trainer.fit(model, tr, dev_eval, te_eval, batch_size=32, grad_accum_cnt=1, learning_rate=5e-4, max_epochs=20,
                    model_file=os.path.join(log_dir, f'BiGCN_{test_event_name}.pkl'))

     1  [  0 |  20], loss/acc = 0.96027029/0.4687500, loss_mean/std=0.9602703/0.0000000, best_valid_acc:0.0000000 
     2  [  0 |  20], loss/acc = 0.67890692/0.5937500, loss_mean/std=0.8195886/0.1406817, best_valid_acc:0.0000000 
     3  [  0 |  20], loss/acc = 0.81773812/0.4062500, loss_mean/std=0.8189718/0.1148694, best_valid_acc:0.0000000 
     4  [  0 |  20], loss/acc = 0.77237481/0.4062500, loss_mean/std=0.8073225/0.1015054, best_valid_acc:0.0000000 
     5  [  0 |  20], loss/acc = 0.67834967/0.5312500, loss_mean/std=0.7815280/0.1044228, best_valid_acc:0.0000000 
     6  [  0 |  20], loss/acc = 0.54667854/0.6875000, loss_mean/std=0.7423864/0.1294105, best_valid_acc:0.0000000 
     7  [  0 |  20], loss/acc = 0.56937867/0.6875000, loss_mean/std=0.7176710/0.1342376, best_valid_acc:0.0000000 
     8  [  0 |  20], loss/acc = 0.76326668/0.5937500, loss_mean/std=0.7233705/0.1264700, best_valid_acc:0.0000000 
     9  [  0 |  20], loss/acc = 0.70110166/0.5625000, loss_mean/std=0.7208962/0.

    73  [  0 |  20], loss/acc = 0.40070173/0.8125000, loss_mean/std=0.4869226/0.0724959, best_valid_acc:0.0000000 
    74  [  0 |  20], loss/acc = 0.61606038/0.7812500, loss_mean/std=0.4879267/0.0741202, best_valid_acc:0.0000000 
    75  [  0 |  20], loss/acc = 0.42201242/0.7812500, loss_mean/std=0.4903195/0.0711205, best_valid_acc:0.0000000 
    76  [  0 |  20], loss/acc = 0.57789695/0.7187500, loss_mean/std=0.4988025/0.0709420, best_valid_acc:0.0000000 
    77  [  0 |  20], loss/acc = 0.51261342/0.7500000, loss_mean/std=0.4978416/0.0706176, best_valid_acc:0.0000000 
    78  [  0 |  20], loss/acc = 0.60892594/0.6250000, loss_mean/std=0.5104557/0.0666923, best_valid_acc:0.0000000 
    79  [  0 |  20], loss/acc = 0.53320283/0.7187500, loss_mean/std=0.5118612/0.0668604, best_valid_acc:0.0000000 
    80  [  0 |  20], loss/acc = 0.50077868/0.8437500, loss_mean/std=0.5099047/0.0665830, best_valid_acc:0.0000000 
    81  [  0 |  20], loss/acc = 0.32295573/0.9375000, loss_mean/std=0.5012324/0.

   142  [  1 |  20], loss/acc = 0.34169060/0.8750000, loss_mean/std=0.4625954/0.1069852, best_valid_acc:0.8385543 
   143  [  1 |  20], loss/acc = 0.38427112/0.8125000, loss_mean/std=0.4570270/0.1080138, best_valid_acc:0.8385543 
   144  [  1 |  20], loss/acc = 0.40550631/0.8437500, loss_mean/std=0.4495813/0.1061601, best_valid_acc:0.8385543 
   145  [  1 |  20], loss/acc = 0.41073918/0.8125000, loss_mean/std=0.4560630/0.0994191, best_valid_acc:0.8385543 
   146  [  1 |  20], loss/acc = 0.60780507/0.7187500, loss_mean/std=0.4497284/0.0843744, best_valid_acc:0.8385543 
   147  [  1 |  20], loss/acc = 0.52002382/0.7500000, loss_mean/std=0.4424108/0.0704681, best_valid_acc:0.8385543 
   148  [  1 |  20], loss/acc = 0.47311622/0.8437500, loss_mean/std=0.4485751/0.0674266, best_valid_acc:0.8385543 
   149  [  1 |  20], loss/acc = 0.77859104/0.6875000, loss_mean/std=0.4700807/0.0950180, best_valid_acc:0.8385543 
   150  [  1 |  20], loss/acc = 0.41127217/0.7812500, loss_mean/std=0.4645912/0.

   211  [  1 |  20], loss/acc = 0.37864447/0.8125000, loss_mean/std=0.4045160/0.0868240, best_valid_acc:0.8385543 
   212  [  1 |  20], loss/acc = 0.28490561/0.8750000, loss_mean/std=0.3887122/0.0779361, best_valid_acc:0.8385543 
   213  [  1 |  20], loss/acc = 0.38767707/0.8125000, loss_mean/std=0.3899252/0.0777215, best_valid_acc:0.8385543 
   214  [  1 |  20], loss/acc = 0.61130404/0.7812500, loss_mean/std=0.4045295/0.0895980, best_valid_acc:0.8385543 
   215  [  1 |  20], loss/acc = 0.34676421/0.8750000, loss_mean/std=0.3930406/0.0811373, best_valid_acc:0.8385543 
   216  [  1 |  20], loss/acc = 0.63752717/0.6875000, loss_mean/std=0.4051778/0.0970798, best_valid_acc:0.8385543 
   217  [  1 |  20], loss/acc = 0.25673723/0.9062500, loss_mean/std=0.3973161/0.1022768, best_valid_acc:0.8385543 
   218  [  1 |  20], loss/acc = 0.32388610/0.9375000, loss_mean/std=0.3921757/0.1032500, best_valid_acc:0.8385543 
   219  [  1 |  20], loss/acc = 0.41961208/0.7500000, loss_mean/std=0.4006348/0.

   283  [  2 |  20], loss/acc = 0.31374189/0.8437500, loss_mean/std=0.3240707/0.0982531, best_valid_acc:0.8385543 
   284  [  2 |  20], loss/acc = 0.26833943/0.8750000, loss_mean/std=0.3219552/0.0989721, best_valid_acc:0.8385543 
   285  [  2 |  20], loss/acc = 0.23890936/0.9062500, loss_mean/std=0.3226313/0.0983543, best_valid_acc:0.8385543 
   286  [  2 |  20], loss/acc = 0.16936165/0.9687500, loss_mean/std=0.3125169/0.1030811, best_valid_acc:0.8385543 
   287  [  2 |  20], loss/acc = 0.22386619/0.8750000, loss_mean/std=0.3117393/0.1036865, best_valid_acc:0.8385543 
   288  [  2 |  20], loss/acc = 0.23193333/0.8750000, loss_mean/std=0.3091408/0.1049945, best_valid_acc:0.8385543 
   289  [  2 |  20], loss/acc = 0.33607039/0.7812500, loss_mean/std=0.3010822/0.0960523, best_valid_acc:0.8385543 
   290  [  2 |  20], loss/acc = 0.18370643/0.9062500, loss_mean/std=0.2953891/0.0994076, best_valid_acc:0.8385543 
   291  [  2 |  20], loss/acc = 0.34547108/0.8125000, loss_mean/std=0.2965619/0.

KeyboardInterrupt: 